In [1]:
# API KEY: sk-82F0fUi3hSi_drnaRG9Z72jEbFbIqPdYHHClASd7kET3BlbkFJJqxRZ40hP-p3XxjBMCQK3qUT40AmziQLi9Vz5WIcIA
# ORG ID: org-rjJCwF3sTsfOnw3I2Eaj4zdP
import json
import os
import re
import openai
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
def preprocess(data):
    no_punctuation_and_specials = re.sub(r'[^\w\s]', '', str(data))
    tokens = word_tokenize(str(no_punctuation_and_specials))
    filtered_tokens = [str(word) for word in tokens if word.lower() not in stop_words]

    return " ".join(filtered_tokens)

In [3]:
load_dotenv()
openai.api_key = os.getenv("API_KEY")

In [4]:
df = pd.read_csv('../data/train.csv')
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [5]:
toxic_classification = df[['comment_text', 'toxic']]
toxic_classification['comment_text'] = toxic_classification['comment_text'].apply(preprocess)
toxic_classification.columns = ['Text', 'Toxic']
toxic_classification

C:\Users\dodsone\AppData\Local\Temp\ipykernel_18016\259305162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic_classification['comment_text'] = toxic_classification['comment_text'].apply(preprocess)


,Text,Toxic
0,Explanation edits made username Hardcore Metal...,0
1,Daww matches background colour Im seemingly st...,0
2,Hey man Im really trying edit war guy constant...,0
3,cant make real suggestions improvement wondere...,0
4,sir hero chance remember page thats,0
...,...,...
159566,second time asking view completely contradicts...,0
159567,ashamed horrible thing put talk page 128611993,0
159568,Spitzer Umm theres actual article prostitution...,0
159569,looks like actually put speedy first version d...,0


In [6]:
training_json = []

In [7]:
for index, row in toxic_classification.iterrows():
    training_json.append({
        "messages" : [
            {
                "role": "system", 
                "content": "Your job is to detect toxicity from a non-biased viewpoint a label prompted content as either toxic or non-toxic."
            },
            {
                "role": "user",
                "content": row['Text']
            },
            {
                "role": "assistant",
                "content": str(row['Toxic'])
            },
        ]
    })

In [8]:
with open('../data/kaggle_toxic_classification_dataset.jsonl', 'w') as f:
    for entry in training_json:
        f.write(f"{json.dumps(entry)}\n")

In [9]:
client = OpenAI(
    api_key=openai.api_key
)

In [10]:
client.files.create(
  file=open("../data/kaggle_toxic_classification_dataset.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-gYnCm7EvYy3qnTJhpMJ5sNs7', bytes=78420036, created_at=1727915595, filename='kaggle_toxic_classification_dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [11]:
client.files.list()

SyncPage[FileObject](data=[FileObject(id='file-gYnCm7EvYy3qnTJhpMJ5sNs7', bytes=78420036, created_at=1727915595, filename='kaggle_toxic_classification_dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-sw9JIkTReL2JanwwHKAhNuiG', bytes=48420688, created_at=1727913270, filename='training.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)], object='list', has_more=False, first_id='file-gYnCm7EvYy3qnTJhpMJ5sNs7', last_id='file-sw9JIkTReL2JanwwHKAhNuiG')

In [12]:
client.fine_tuning.jobs.create(
    training_file="file-gYnCm7EvYy3qnTJhpMJ5sNs7",
    model="gpt-4o-2024-08-06"
)

FineTuningJob(id='ftjob-24RkNBLAynipz5qQ0pUOnAQN', created_at=1727915646, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-S652ryxRoR0nJQpnMy7g1xcm', result_files=[], seed=842925592, status='validating_files', trained_tokens=None, training_file='file-gYnCm7EvYy3qnTJhpMJ5sNs7', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [13]:
client.fine_tuning.jobs.retrieve("ftjob-24RkNBLAynipz5qQ0pUOnAQN")

FineTuningJob(id='ftjob-24RkNBLAynipz5qQ0pUOnAQN', created_at=1727915646, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-S652ryxRoR0nJQpnMy7g1xcm', result_files=[], seed=842925592, status='validating_files', trained_tokens=None, training_file='file-gYnCm7EvYy3qnTJhpMJ5sNs7', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
client = OpenAI(
    api_key=openai.api_key
)
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", 
            "content": "Your job is to detect toxicity from a non-biased viewpoint a label prompted content as either toxic or non-toxic."
        },
        {
            "role": "user",
            "content": "Fuck you you fucking bitch"
        }
    ]
)

In [ ]:
print(completion.choices[0].message.content)

Toxic
